# Box Office - dataframe 

This notebook is used for creating a dataframe from the webscraped box office `csv`-files. Finally, we save it as a pickle for easier access in other notebook.

In [1]:
import pandas as pd
import numpy as np
import os
import glob

Load all `csv`-files of the chosen webscraped movies. Saved in a dictionary at first.

In [2]:
daily_dir = f"data{os.sep}time_series_box_office_data{os.sep}"

path = os.getcwd()
path = os.path.join(path, daily_dir)
csv_files = glob.glob(os.path.join(path, '*.csv'))

movie_names = np.empty(len(csv_files),dtype='<U100')
daily = {}

for i,f in enumerate(csv_files):
    #Get the keys for our dictionary
    movie_names[i] = f[f.find('office_data'):]
    movie_names[i] = movie_names[i][movie_names[i].find(f'{os.sep}'):]
    movie_names[i] = movie_names[i][:movie_names[i].find('___')]
    movie_names[i] = movie_names[i].replace(f'{os.sep}','')
    # read the csv file
    daily[movie_names[i]] = pd.read_csv(f)
    daily[movie_names[i]] = daily[movie_names[i]].drop("Unnamed: 0", axis=1)


We choose to restructure the dictionary as a single-indexed `pandas` dataframe to easily access `pandas` tools and for compatibility with the remaining data sets we use (like Quotebank and IMDb).

In [3]:
# restructure dictionary into accesible single-indexed dataframe
df_boxOffice = pd.concat(daily)
df_boxOffice.index.names = ['movie', 'index']
df_boxOffice = df_boxOffice.reset_index(['movie'])
df_boxOffice.index = np.arange(len(df_boxOffice))
df_boxOffice['movie'] = [(" ").join(movie.split("_")) for movie in df_boxOffice.movie] # compatibility with Quotebank and IMDb dataframes
df_boxOffice

,movie,days,dow,rank,daily,theaters,special events
0,Aladdin,2019-05-24,Friday,1,31358935.0,4476,NaN
1,Aladdin,2019-05-25,Saturday,1,30013295.0,4476,NaN
2,Aladdin,2019-05-26,Sunday,1,30128699.0,4476,NaN
3,Aladdin,2019-05-27,Monday,1,25305033.0,4476,Memorial Day
4,Aladdin,2019-05-28,Tuesday,1,12014982.0,4476,NaN
...,...,...,...,...,...,...,...
6956,Zootopia,2016-07-31,Sunday,43,2144.0,24,NaN
6957,Zootopia,2016-08-01,Monday,43,845.0,24,NaN
6958,Zootopia,2016-08-02,Tuesday,43,904.0,24,NaN
6959,Zootopia,2016-08-03,Wednesday,45,865.0,24,NaN


We are aware that the movie title in this dataframe is currently named from the naming of the `csv`-file it comes from. This is a problem as this title is not similar to the one used in the `movie` attribute in the Quotebank data set. See below that i.e. Star Wars does not include the sub-title of the Star Wars episode.

In [4]:
boxOffice_titles = df_boxOffice.movie.unique()
boxOffice_titles.sort()
boxOffice_titles

array(['Aladdin', 'Aquaman', 'Avengers Age of Ultron', 'Avengers Endgame',
       'Avengers Infinity War', 'Bad Boys for Life',
       'Batman v Superman Dawn of Justice', 'Beauty and the Beast',
       'Birds of Prey', 'Black Panther', 'Bohemian Rhapsody',
       'Captain America Civil War', 'Captain Marvel', 'Deadpool',
       'Deadpool 2', 'Despicable Me 3', 'Dolittle',
       'Fantastic Beasts The Crimes of Grindelwald',
       'Fantastic Beasts and Where to Find Them', 'Finding Dory',
       'Frozen II', 'Furious 7', 'Guardians of the Galaxy Vol. 2',
       'Incredibles 2', 'Inside Out', 'Joker', 'Jumanji The Next Level',
       'Jumanji Welcome to the Jungle', 'Jurassic World',
       'Jurassic World Fallen Kingdom', 'Minions',
       'Mission Impossible Fallout', 'Mission Impossible Rogue Nation',
       'Onward', 'Rogue One A Star Wars Story', 'Sonic the Hedgehog',
       'Spectre', 'Spider Man Far from Home', 'Spider Man Homecoming',
       'Star Wars Episode IX', 'Star Wars E

To solve this issue of different representation of movie files, we create an automatic mapping that simply compares a stripped version (of hyphans and colons) of the Quotebank titles to the box office titles. We see that this approach is succesfull for almost every title - but there are a few exceptions that we manually handle. We first load the Quotebank data to see how movies are named in that data frame. 

In [5]:
data_dir = os.getcwd() + os.sep + 'data'
df_Quotebank = pd.read_pickle(rf"{data_dir}{os.sep}Quotebank_full.pkl") 

In [6]:
quotebank_titles = df_Quotebank.movie.unique()
quotebank_titles.sort()
quotebank_titles

array(['Aladdin', 'Aquaman', 'Avengers: Age of Ultron',
       'Avengers: Endgame', 'Avengers: Infinity War', 'Bad Boys for Life',
       'Batman v Superman: Dawn of Justice', 'Beauty and the Beast',
       'Birds of Prey: And the Fantabulous Emancipation of One Harley Quinn',
       'Black Panther', 'Bohemian Rhapsody', 'Captain America: Civil War',
       'Captain Marvel', 'Deadpool', 'Deadpool 2', 'Despicable Me 3',
       'Dolittle', 'Fantastic Beasts and Where to Find Them',
       'Fantastic Beasts: The Crimes of Grindelwald', 'Fast & Furious 7',
       'Finding Dory', 'Frozen II', 'Guardians of the Galaxy Vol. 2',
       'Incredibles 2', 'Inside Out', 'Joker', 'Jumanji: The Next Level',
       'Jumanji: Welcome to the Jungle', 'Jurassic World',
       'Jurassic World: Fallen Kingdom', 'Minions',
       'Mission: Impossible - Fallout',
       'Mission: Impossible - Rogue Nation', 'Onward', 'Rogue One',
       'Sonic the Hedgehog', 'Spectre', 'Spider-Man: Far from Home',
       'S

Then we do the automatic mapping and print the Quotebank titles that could not be mapped automatically.

In [7]:
#This cell will get the titles and more to be examined (tm2e) by comparing the titles in the quotebank data and the daily gross data.
#Furthermore, it will also keep both the title versions. The daily as keys and the quotebank as items.

title_mapping = {}
exceptions = []

for Quotebank_title in quotebank_titles:
    Quotebank_edit= Quotebank_title.replace(':','')
    if '-' in Quotebank_edit:
        Quotebank_edit = Quotebank_edit[:Quotebank_edit.find(' - ')]
        
    if np.any(boxOffice_titles == Quotebank_edit):
        for boxOffice_title in boxOffice_titles:
            if Quotebank_edit == boxOffice_title:
                title_mapping[Quotebank_edit] = Quotebank_title
    else:
        exceptions.append(Quotebank_title)
        
print(f"Failed to map {len(exceptions)} movie titles. They were:\n")
for exception in exceptions:
    print(f"Quotebank title: {exception}")

Failed to map 7 movie titles. They were:

Quotebank title: Birds of Prey: And the Fantabulous Emancipation of One Harley Quinn
Quotebank title: Fast & Furious 7
Quotebank title: Mission: Impossible - Fallout
Quotebank title: Mission: Impossible - Rogue Nation
Quotebank title: Rogue One
Quotebank title: Spider-Man: Far from Home
Quotebank title: Spider-Man: Homecoming


In [8]:
boxOffice_titles

array(['Aladdin', 'Aquaman', 'Avengers Age of Ultron', 'Avengers Endgame',
       'Avengers Infinity War', 'Bad Boys for Life',
       'Batman v Superman Dawn of Justice', 'Beauty and the Beast',
       'Birds of Prey', 'Black Panther', 'Bohemian Rhapsody',
       'Captain America Civil War', 'Captain Marvel', 'Deadpool',
       'Deadpool 2', 'Despicable Me 3', 'Dolittle',
       'Fantastic Beasts The Crimes of Grindelwald',
       'Fantastic Beasts and Where to Find Them', 'Finding Dory',
       'Frozen II', 'Furious 7', 'Guardians of the Galaxy Vol. 2',
       'Incredibles 2', 'Inside Out', 'Joker', 'Jumanji The Next Level',
       'Jumanji Welcome to the Jungle', 'Jurassic World',
       'Jurassic World Fallen Kingdom', 'Minions',
       'Mission Impossible Fallout', 'Mission Impossible Rogue Nation',
       'Onward', 'Rogue One A Star Wars Story', 'Sonic the Hedgehog',
       'Spectre', 'Spider Man Far from Home', 'Spider Man Homecoming',
       'Star Wars Episode IX', 'Star Wars E

These exceptions are handled manually so we look at the previously printed Box Office movie titles to find the connection.

In [9]:
boxOffice_exception = ['Birds of Prey', 'Furious 7', 
                       'Rogue One A Star Wars Story', 
                       'Spider Man Far from Home', 'Spider Man Homecoming',
                       'Mission Impossible Fallout', 'Mission Impossible Rogue Nation']

for i in range(len(exceptions)):
    title_mapping[boxOffice_exception[i]] = exceptions[i]

We check it by seeing if the values and keys are equal to the Quotebank and box office titles resepectively.

In [10]:
print(f"Number of movies in Box Office data: {boxOffice_titles.__len__()}")
print(f"Number of movies in Quotebank data: {quotebank_titles.__len__()}")

diff = np.setdiff1d(boxOffice_titles, list(title_mapping.keys()))
diff

Number of movies in Box Office data: 58
Number of movies in Quotebank data: 56


array(['The Call of the Wild', 'Tolo Tolo'], dtype=object)

This means that some of the box office titles (the two above) are not occuring in the Quotebank data. We find it evident to leave these ones out and not incorporate them in the mapping as they do not occur in our Quotebank data. Therefore we drop these rows from the Box Office dataframe.

In [11]:
removal_condition = np.logical_or(df_boxOffice.movie == diff[0], df_boxOffice.movie == diff[1])
df_boxOffice = df_boxOffice[-removal_condition]
df_boxOffice.index = np.arange(len(df_boxOffice))

In [12]:
df_boxOffice.movie.unique().__len__()

56

In [13]:
df_boxOffice.shape

(6905, 7)

 Checking the difference to the mapping in quotebank titles is somewhat more relevant.

In [14]:
np.setdiff1d(quotebank_titles, list(title_mapping.values()))

array([], dtype=object)

As the difference between the Quotebank data and the values in the mapping dictionary is the empty set we know that we have accounted for all possible mappings from Box Office titles to Quotebank titles. We apply the mapping to the Box Office dataframe.

In [15]:
df_boxOffice = df_boxOffice.rename(columns={'movie': 'old_title'})
df_boxOffice['movie'] = [title_mapping[BO_title] for BO_title in df_boxOffice.old_title]

df_boxOffice = df_boxOffice.drop('old_title', axis=1)

And a final check to see if the titles are now coherent.

In [16]:
set(df_boxOffice.movie.unique()) == set(quotebank_titles)

True

We save the dataframe as a pickle.

In [17]:
filename = f"boxOffice"
data_dir = os.getcwd() + os.sep + 'data'

df_boxOffice.to_pickle(rf"{data_dir}{os.sep}{filename}.pkl")

print(f"Dataframe was pickled and saved to directory:\n{data_dir} \n")
print(f"Shape of dataframe: {df_boxOffice.shape}")

Dataframe was pickled and saved to directory:
C:\Users\Albert Kjøller\Documents\EPFL\Courses\CS-401_ADA\ada-2021-project-f-jab\exploratory\data 

Shape of dataframe: (6905, 7)
